# Priprema podataka za izradu modela predviđanja preživljavanja putnika na brodu Titanik

Ovo je primer primene različitih tehnika za pripremu podataka za izradu prediktivnih modela. U primeru se koristi skup podataka o putnicima broda Titanik, a skup se koristi za izradu modela predviđanja njihovog preživljavanja. 

U pripremi se primenjuju sledeće aktivnosti: analiza i imputacija nedostajućih vrednosti, transformacija tipova, inženjering veličina, kodiranje nominalnih veličina, podela na podskupove za treniranje i testiranje i normalizacija. Nakon svih aktivnosti pripreme, izvršeno je treniranje modela koji koristi algoritam logističke regresije. 

In [982]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [983]:
df=pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Model bez pripreme podataka

Pre aktivnosti pripreme podataka, formiran je model koji se zasniva na podacima kod kojih je primenjena samo osnovna priprema, neophodna za treniranje modela. Naime, model je nemoguće trenirati skupom podataka u kojem ima nedostajućih podataka i podataka tekstualnog tipa. Dakle, primenjene su aktivnosti proste imputacije nedostajućih podataka (imputiranje srednje vrednosti) i kodiranje labelama nominalnih veličina. Iz skupa su uklonjene veličine kod kojih postoji veliki broj nedostajućih podataka i one čiji je značaj, odnosno uticaj na izlaznu veličinu neodređen.

In [984]:
dfd=pd.read_csv('train.csv')
dfd=dfd.drop('PassengerId',1)
dfd=dfd.drop('Ticket', 1)
dfd=dfd.drop('Cabin', 1)
dfd=dfd.drop('Name', 1)

dfd.loc[np.isnan(dfd['Age']),'Age']=dfd['Age'].mean()
dfd['Embarked']=dfd['Embarked'].fillna(dfd['Embarked'].mode()[0])

gle = LabelEncoder()
dfd['Sex'] = gle.fit_transform(dfd['Sex'])
dfd['Embarked'] = gle.fit_transform(dfd['Embarked'])

Xd=dfd.values[:,1:8]
yd=dfd.values[:,0]

from sklearn.model_selection import train_test_split
X_traind, X_testd, y_traind, y_testd=train_test_split(Xd,yd,test_size=0.3)

modeld=LogisticRegression()
modeld.fit(X_traind, y_traind)
accuracyd=modeld.score(X_testd, y_testd)
print('Tačnost modela je:'+str(accuracyd))

Tačnost modela je:0.7723880597014925


## Analiza nedostajućih podataka i imputacija

Najpre se vrši analiza i imputacija nedostajućih podataka.

In [985]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Uočava se veliki broj nedostajućih podataka veličine Cabin, značajan broj nedostajućih podataka veličine Age i 2 nedostajuća podatka veličine Embarked. Kako je značaj veličine Cabin neodređen, ona se uklanja iz skupa podataka. Uklanjaju se i veličine PassengerId i Ticket jer je njihov značaj neodređen ili ne postoji.

In [986]:
df=df.drop('PassengerId',1)
df=df.drop('Ticket', 1)
df=df.drop('Cabin', 1)

Za imputiranje nedostajućih podataka veličine Age, koristi se strategija imputacije slučajnih brojeva u opsegu (prosek-std, prosek+std). Najpre, vrši se analiza distribucije onih podataka za koje nedostaje veličina Age.

In [987]:
df[np.isnan(df['Age'])].describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,177.000000,177.000000,0.0,177.000000,177.000000,177.000000
mean,0.293785,2.598870,NaN,0.564972,0.180791,22.158567
std,0.456787,0.763216,NaN,1.626316,0.534145,31.874608
min,0.000000,1.000000,NaN,0.000000,0.000000,0.000000
25%,0.000000,3.000000,NaN,0.000000,0.000000,7.750000
50%,0.000000,3.000000,NaN,0.000000,0.000000,8.050000
75%,1.000000,3.000000,NaN,0.000000,0.000000,24.150000
max,1.000000,3.000000,NaN,8.000000,2.000000,227.525000


U rezultatima proste statističke analize se uočava da pretežno nedostaju podaci o godinama starosti za putnike koji putuju sami (mean SibSp=0.56, mean Parch=0.18). Zato, za proračun slučajnih brojeva u opsegu (prosek-std, prosek+std) će se koristiti samo podaci o putnicima koji putuju sami, za koje postoji podatak o veličini Age.

In [988]:
import random as rnd

agemean=df['Age'][(df['SibSp']==0) & (df['Parch']==0)].mean()
agestd=df['Age'][(df['SibSp']==0) & (df['Parch']==0)].std()

df.loc[np.isnan(df['Age']),'Age']=round(rnd.uniform(agemean-agestd,agemean+agestd),0)

Potom, istražuje se da li postoje podaci za koje su kontinualne veličine (u ovom slučaju, Fare) jednake nuli. Ovo su mogući slučajevi nedostajućih podataka. Uočeno je da postoji 15 ovakvih slučajeva. Kao i u ranijem slučaju, podaci kod kojih je vrednost cene karte jednaka nuli su prisutni kod putnika koji putuju sami (SibSp=0, Parch=0).

In [989]:
df[df['Fare']==0].describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,15.000000,15.000000,15.000000,15.0,15.0,15.0
mean,0.066667,1.933333,38.800000,0.0,0.0,0.0
std,0.258199,0.798809,7.456541,0.0,0.0,0.0
min,0.000000,1.000000,19.000000,0.0,0.0,0.0
25%,0.000000,1.000000,38.500000,0.0,0.0,0.0
50%,0.000000,2.000000,42.000000,0.0,0.0,0.0
75%,0.000000,2.500000,42.000000,0.0,0.0,0.0
max,1.000000,3.000000,49.000000,0.0,0.0,0.0


Ovi podaci se proglašavaju nedostajućim.

In [990]:
df.loc[df['Fare']==0,'Fare']=np.nan

Ranije primenjena strategija imputacije nedostajućih podataka se primenjuje i u ovom slučaju.

In [991]:
faremean=df['Fare'][(df['SibSp']==0) & (df['Parch']==0)].mean()
farestd=df['Fare'][(df['SibSp']==0) & (df['Parch']==0)].std()

df.loc[np.isnan(df['Fare']),'Fare']=round(rnd.uniform(faremean-farestd,faremean+farestd),0)

Imputiraju se nedostajuće vrednosti veličine Embarked. Ova veličina je nominalna - za imputaciju se koriste vrednosti najčešćeg pojavljivanja, odnosno mode().

In [992]:
emode=df['Embarked'].mode()[0]
df['Embarked']=df['Embarked'].fillna(emode)

In [993]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Fare      891 non-null    float64
 8   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


## Inženjering veličina

Uvidom u vrednosti veličine Name, može se zaključiti da ono sadrži dve informacije od potencijalne važnosti za model. Prvo, podaci ove veličine sadrže titule osoba. Drugo, moguće je među putnicima uočiti sve supruge i to na osnovu pravila upisivanja imena supruga - njihovo devojačko ime se navodi u zagradi. Dakle, prvi korak u inžinjeringu veličina je ekstrakcija titula i podatka da li je osoba supruga ili ne. Nakon ekstrakcije, briše se veličina Name

In [994]:
def getTitle(name):
    f=name.split('. ')[0]
    if f:
        d=f.split(', ')[1]
        if d:
            return d
        return ''
    return ''

def isWife(name):
    f=name.split('(')
    if(f[0]==name):
        return 0
    else:
        return 1

df['Title']=df['Name'].apply(getTitle)
df['IsWife']=df['Name'].apply(isWife)
df['FamilySize']=df['SibSp']+df['Parch']+1
df['IsAlone']=0
df.loc[df['FamilySize']==1,'IsAlone']=1

df['Title']=df['Title'].replace('Mlle','Miss')
df['Title']=df['Title'].replace('Ms','Miss')
df['Title']=df['Title'].replace('Mme','Mrs')
df['Title'] = df['Title'].replace(['Lady', 'the Countess', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df['Title']=df['Title'].astype('str')

df=df.drop('Name',1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsWife,FamilySize,IsAlone
0,0,3,male,22.0,1,0,7.2500,S,Mr,0,2,0
1,1,1,female,38.0,1,0,71.2833,C,Mrs,1,2,0
2,1,3,female,26.0,0,0,7.9250,S,Miss,0,1,1
3,1,1,female,35.0,1,0,53.1000,S,Mrs,1,2,0
4,0,3,male,35.0,0,0,8.0500,S,Mr,0,1,1


## Transformacija tipova

Vrši se transformacija tipova odgovarajućih veličina.

In [995]:
df['Age']=df['Age'].astype('int')
df['Fare']=df['Fare'].astype('int')
df['Sex']=df['Sex'].astype('str')
df['Title']=df['Title'].astype('str')

## Kodiranje nominalnih veličina

Konačno, vrši se kodiranje nominalnih veličina.

In [996]:
gle = LabelEncoder()
df['Sex'] = gle.fit_transform(df['Sex'])
df['Embarked'] = gle.fit_transform(df['Embarked'])
df['Title'] = gle.fit_transform(df['Title'])

In [997]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsWife,FamilySize,IsAlone
0,0,3,1,22,1,0,7,2,2,0,2,0
1,1,1,0,38,1,0,71,0,3,1,2,0
2,1,3,0,26,0,0,7,2,1,0,1,1
3,1,1,0,35,1,0,53,2,3,1,2,0
4,0,3,1,35,0,0,8,2,2,0,1,1


## Podela skupa podataka na skup za treniranje i skup za testiranje

Vrši se podela skupa podataka na skup za treniranje i skup za testiranje.

In [998]:
X=df.values[:,1:12]
y=df.values[:,0]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3)

## Normalizacija podataka

Vrši se normalizacija.

In [999]:
mms=MinMaxScaler(feature_range=(0, 1))
scaler = mms.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Treniranje i validacija prediktivnog modela

Koristi se algoritam logističke regresije. Model se trenira bez podešavanja hiper parametara, odnosno izrađuje se neoptimizovan model. Njegova svrha je samo demonstracija tačnosti modela sa primenjenim tehnikama pripreme podataka.

In [1000]:
model=LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [1001]:
accuracy=model.score(X_test, y_test)
print('Tačnost modela je:'+str(accuracy))
print('Tačnost nepripremljenog modela je:'+str(accuracyd))

Tačnost modela je:0.8470149253731343
Tačnost nepripremljenog modela je:0.7723880597014925
